<a href="https://colab.research.google.com/github/langwnwk11/quant/blob/main/fund_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 基金相关数据获取
github定时任务每天早上8点定时执行获取因子和基金的数据



# 依赖相关

## 安装依赖库

In [1]:
!pip install -q akshare --upgrade
!pip install -q baostock -i https://pypi.org/simple
!pip install --upgrade baostock -i https://pypi.org/simple
!pip install -q yfinance
!pip install -q pyarrow

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 120.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 2.0 MB/s eta 0:00:00


## 导入公共包

In [2]:

import yfinance as yf
import pandas as pd
import numpy as np
import akshare as ak
import baostock as bs
from datetime import datetime
import os
# from google.colab import drive
import time # Import the time module
# drive.mount('/content/drive')

#全局配置

##全局变量

### 常量

In [17]:
start_date = '20160101'
end_date = datetime.now().strftime('%Y%m%d')

skip=0

path = '/content/drive/MyDrive/QuantData/fund'

attemp_count=3
retry_delay_seconds=60 # New global variable for retry delay
post_execution_delay_seconds=10 # New global variable for delay after each execution

### 基金代码

In [18]:
fund_list =[
    {"code":"001437","name":"易方达瑞享混合I"},
    {"code":"018484","name":"财通资管医疗保健混合A"},
    {"code":"001915","name":"宝盈医疗健康沪港深股票A"},
    {"code":"020972","name":"易方达国证机器人产业etf A"},
    {"code":"018344","name":"华夏中证机器人etf A"},
    {"code":"016563","name":"金鹰红利价值混合C"},
    {"code":"005269","name":"华泰柏瑞港股通量化混合A"},
    {"code":"001917","name":"招商量化精选股票A"},
    {"code":"019684","name":"万家添利A"},
    {"code":"008923","name":"建信医疗健康行业股票A"},
    {"code":"001230","name":"鹏华医药科技股票A"},
    {"code":"008763","name":"天弘越南市场股票(QDII)A"},
    {"code":"006105","name":"宏利印度股票(QDII)A"},
    {"code":"007549","name":"中泰开阳价值优选混合A"},
    {"code":"006624","name":"中泰玉衡价值优选混合A"},
    {"code":"121010","name":"国投瑞银瑞源灵活配置A"},
    {"code":"007300","name":"国联中证半导体产品与设备A"},
    {"code":"001167","name":"金鹰科技创新股票A"},
    {"code":"004496","name":"前海开源多策略混合A"},
    {"code":"110017","name":"易方达增强回报债券A"},
    {"code":"260112","name":"景顺长城能源基建混合A"},
    {"code":"160224","name":"国泰中证计算机etfA"},
    {"code":"090019","name":"大成景恒混合A"},
    {"code":"006195","name":"国金量化多因子A"},
    {"code":"001028","name":"华安物联网主题股票A"},
    {"code":"018653","name":"万家国证2000指数增强A"},
    {"code":"010387","name":"易方达医药生物A"},
    {"code":"007340","name":"南方科技创新混合A"},
    {"code":"007355","name":"汇添富科技创新混合A"},
    {"code":"163822","name":"中银主题策略混合A"},
    {"code":"001552","name":"天弘中证证券保险指数A"},
    {"code":"002602","name":"易方达丰惠混合"},
    {"code":"001927","name":"华夏消费升级混合A"},
    {"code":"519766","name":"交银荣鑫灵活配置混合A"},
    {"code":"001323","name":"东吴移动互联混合A"},
    {"code":"008715","name":"景顺长城价值配置混合"},
    {"code":"001672","name":"国寿安保智慧生活股票A"},
    {"code":"008404","name":"华泰紫金泰盈混合A"},
    {"code":"007792","name":"嘉实央企创新驱动ETF A"},
    {"code":"270028","name":"广发制造业精选混合A"},
    {"code":"020011","name":"国泰沪深300指数A"},
    {"code":"013292","name":"富国中证智能汽车Lof C"},
    {"code":"000893","name":"工银创新动力股票"},
    {"code":"012724","name":"国泰中证畜牧养殖ETF A"},
    {"code":"006479","name":"广发纳斯达克100 etf C"},
    {"code":"021216","name":"华宝中证A50ETF A"},
    {"code":"000974","name":"安信消费医药主题A"},
    {"code":"501011","name":"汇添富中证中药etf A"},

      ]




### 因子

In [19]:
factor_list=[{'code':'sh000300','name':'沪深300'},

      # {'code':'510500','name':'中证500ETF'},
      # {'code':'588000','name':'科创50ETF'}, ## 上交所

      # ##深交所
      #  {'code':'159922','name':'中证500ETF'},
      # {'code':'159915','name':'创业板ETF'},
      # {'code':'159901','name':'深证100ETF'}
      ]

## 定于公共函数

### 重试装饰器

In [20]:
def retry_on_failure(
    func=None,
    attemp_count_param=None,
    retry_delay_seconds_param=None,
    post_execution_delay_seconds_param=None
):
    """一个通用的装饰器，用于实现函数的重试逻辑。"""
    def decorator(f):
        def wrapper(*args, **kwargs):
            # Use provided parameters or global defaults
            current_attemp_count = attemp_count_param if attemp_count_param is not None else attemp_count
            current_retry_delay_seconds = retry_delay_seconds_param if retry_delay_seconds_param is not None else retry_delay_seconds
            current_post_execution_delay_seconds = post_execution_delay_seconds_param if post_execution_delay_seconds_param is not None else post_execution_delay_seconds

            for attempt in range(current_attemp_count):
                try:
                    result = f(*args, **kwargs)
                    time.sleep(current_post_execution_delay_seconds)
                    return result
                except Exception as e:
                    print(f"Attempt {attempt + 1} failed for {f.__name__} with args {args}, kwargs {kwargs}: {e}")
                    if attempt < current_attemp_count - 1:
                        time.sleep(current_retry_delay_seconds)
                    else:
                        print(f"Failed to execute {f.__name__} after {current_attemp_count} attempts.")
                        raise
        return wrapper

    if func is None:
        return decorator
    else:
        return decorator(func)

### 获取指数收益率 baostock

In [21]:
@retry_on_failure(post_execution_delay_seconds_param=5)
def get_index_baostock(prefix,code):
  lg = bs.login()

  # 详细指标参数，参见“历史行情指标参数”章节；“周月线”参数与“日线”参数不同。
  # 周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg
  rs = bs.query_history_k_data_plus(prefix+'.'+code,
      "date,code,open,high,low,close,peTTM",
      start_date='1997-01-01', end_date='2322-01-01', frequency="d")

  # 打印结果集
  data_list = []
  while (rs.error_code == '0') & rs.next():
      # 获取一条记录，将记录合并在一起
      data_list.append(rs.get_row_data())
  result = pd.DataFrame(data_list, columns=rs.fields)
  bs.logout()
  return result

### 格式化padas数据集

In [22]:
def format_df(df,column_date,column_close,column_return):
  df['date'] = pd.to_datetime(df[column_date]) # Ensure it's datetime
  df = df.set_index('date').sort_index() # Set date as index
  df[column_close] = pd.to_numeric(df[column_close], errors='coerce') # Convert 'close' to numeric
  df = df[column_close].pct_change().dropna().rename(column_return) # Calcul
  return df

### 获取申万一级行业信息

In [23]:
@retry_on_failure
def get_sw_first_info():
  df=ak.sw_index_first_info()
  return df

### 获取申万一级具体行业数据


In [24]:
@retry_on_failure
def get_sw_first_induesty_data(code):
  df=ak.index_hist_sw(symbol=code, period="day")
  return df


### 黄金数据获取

In [25]:
@retry_on_failure(post_execution_delay_seconds_param=10)
def get_gold(code):
  df= ak.spot_hist_sge(symbol=code)
  return df

### 基金数据获取

In [26]:
@retry_on_failure(post_execution_delay_seconds_param=10)
def get_fund_data(code,column_name):
  df = ak.fund_open_fund_info_em(symbol=code, indicator=column_name)
  return df

### 获取a股所有的上市公司代码

In [27]:
@retry_on_failure(post_execution_delay_seconds_param=20)
def get_all_stock():
  df = ak.stock_info_a_code_name()
  return df

### 定义线性回归函数

In [28]:

# # 1. 准备数据 (假设你已合并好 df_merged)
# # y: 基金超额收益率 (基金日收益 - 无风险利率，简单起见可直接用日收益率)
# # X: 因子矩阵 (市场、黄金、商品)

# def factor_regression(fund_name, y, X_factors):
#     # 为自变量添加常数项 (即 Alpha 截距)
#     X = sm.add_constant(X_factors)

#     # 构建 OLS 回归模型
#     model = sm.OLS(y, X).fit()

#     print(f"--- {fund_name} 多因子回归报告 ---")
#     print(model.summary())
#     return model

# 数据获取


## 因子数据

### 10年国债收益率

In [ ]:
bond_10 = ak.bond_zh_us_rate(start_date=start_date)
bond_10
rf = bond_10['中国国债收益率10年'].iloc[-1] / 100
print(rf)

0.018193


In [ ]:
bond_10

,日期,中国国债收益率2年,中国国债收益率5年,中国国债收益率10年,中国国债收益率30年,中国国债收益率10年-2年,中国GDP年增率,美国国债收益率2年,美国国债收益率5年,美国国债收益率10年,美国国债收益率30年,美国国债收益率10年-2年,美国GDP年增率
0,2016-01-04,2.4598,2.7488,2.8710,3.6246,0.4112,NaN,1.02,1.73,2.24,2.98,1.22,NaN
1,2016-01-05,2.4944,2.7871,2.8911,3.6446,0.3967,NaN,1.04,1.73,2.25,3.01,1.21,NaN
2,2016-01-06,2.5038,2.7598,2.8861,3.6155,0.3823,NaN,0.99,1.65,2.18,2.94,1.19,NaN
3,2016-01-07,2.4928,2.7538,2.8511,3.5718,0.3583,NaN,0.96,1.61,2.16,2.92,1.20,NaN
4,2016-01-08,2.4790,2.7347,2.8362,3.5537,0.3572,NaN,0.94,1.57,2.13,2.91,1.19,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2687,2026-01-29,1.3908,1.5784,1.8174,2.2800,0.4266,NaN,3.53,3.80,4.24,4.85,0.71,NaN
2688,2026-01-30,1.3765,1.5761,1.8112,2.2890,0.4347,NaN,3.52,3.79,4.26,4.87,0.74,NaN
2689,2026-02-02,1.3852,1.5768,1.8200,2.2810,0.4348,NaN,3.57,3.83,4.29,4.90,0.72,NaN
2690,2026-02-03,1.3772,1.5721,1.8172,2.2800,0.4400,NaN,3.57,3.83,4.28,4.90,0.71,NaN


In [ ]:
# 1 + rf 表示一年的总收益因子。
# **(1/252) 表示将这个总收益因子除以252（通常假设一年有252个交易日），从而得到每天的收益因子。
# - 1 是为了从收益因子中减去本金，只保留每日的收益率。
daily_rf = (1 + rf)**(1/252) - 1  # 假设一年有252个交易日
daily_rf

np.float64(7.154814510212937e-05)

### 沪深300

In [ ]:
df=get_index_baostock('sh','000300')
df_300=format_df(df,'date','close','沪深300')
df_300

login success!
logout success!


,沪深300
date,
2005-01-05,0.009941
2005-01-06,-0.009460
2005-01-07,0.000797
2005-01-10,0.010083
2005-01-11,0.003276
...,...
2026-01-30,-0.009998
2026-02-02,-0.021324
2026-02-03,0.011752


### 中证1000


In [ ]:
# df_1000=get_index_ret('sz000852','中证1000')
# df_1000

df=get_index_baostock('sh','000852')
df
df_1000=format_df(df,'date','close','中证1000')
df_1000

login success!
logout success!


,中证1000
date,
2005-01-05,0.023838
2005-01-06,-0.008009
2005-01-07,0.006241
2005-01-10,0.011760
2005-01-11,0.003440
...,...
2026-01-29,-0.008046
2026-01-30,-0.009283
2026-02-02,-0.033851


### 300价值


In [ ]:
df=get_index_baostock('sh','000919')
df_value=format_df(df,'date','close','300价值')
df_value

login success!
logout success!


,300价值
date,
2008-01-22,-0.076954
2008-01-23,0.045881
2008-01-24,0.003864
2008-01-25,0.010689
2008-01-28,-0.074219
...,...
2026-01-29,0.017391
2026-01-30,-0.010399
2026-02-02,-0.015781


### 300成长

In [ ]:
df=get_index_baostock('sh','000918')
df_growth=format_df(df,'date','close','300成长')
df_growth

login success!
logout success!


,300成长
date,
2008-01-22,-0.075043
2008-01-23,0.040168
2008-01-24,0.006976
2008-01-25,0.015938
2008-01-28,-0.065858
...,...
2026-01-29,0.007002
2026-01-30,-0.006541
2026-02-02,-0.023481


### SMB (规模因子)
用 中证1000指数 (小盘) 减去 沪深300指数 (大盘)

In [ ]:
# 构造 SMB (小盘减大盘)
# 示例：确保日期对齐后再相减
# axis=1：表示将这两个 Series 作为不同的列进行拼接，
# 形成一个 DataFrame。
# join='inner'：这是关键！
# 它确保了只有同时存在于 df_1000 和 df_300 索引（日期）中的数据才会被保留下来。这样，combined_df 中的每一行都对应一个在两个指数中都有数据的日期。
df_1000_300 = pd.concat([df_1000, df_300], axis=1, join='inner')
df_1000_300.columns = ['中证1000', '沪深300']
smb = (df_1000_300['中证1000'] - df_1000_300['沪深300']).rename('SMB')
smb

# # 构造 HML (价值减成长)
# ret_value = get_index_ret("sh000918")  # 沪深300价值
# ret_growth = get_index_ret("sh000919") # 沪深300成长
# hml = ret_value - ret_growth

,SMB
date,
2005-01-05,0.013897
2005-01-06,0.001451
2005-01-07,0.005443
2005-01-10,0.001677
2005-01-11,0.000164
...,...
2026-01-29,-0.015651
2026-01-30,0.000715
2026-02-02,-0.012526


### HML (价值因子)
用 300价值指数 减去 300成长指数。

In [ ]:
df_value_growth = pd.concat([df_value, df_growth], axis=1, join='inner')
df_value_growth.columns = ['300价值', '300成长']
hml = (df_value_growth['300价值'] - df_value_growth['300成长']).rename("HML")
hml

,HML
date,
2008-01-22,-0.001911
2008-01-23,0.005712
2008-01-24,-0.003113
2008-01-25,-0.005249
2008-01-28,-0.008361
...,...
2026-01-29,0.010389
2026-01-30,-0.003859
2026-02-02,0.007701


### 巨潮相关指数
- 大中小盘成长
- 大中小盘价值


In [ ]:
# 大盘成长，价值
df=get_index_baostock('sz','399372')
temp=format_df(df,'date','close','大盘成长')
df=get_index_baostock('sz','399373')
temp1=format_df(df,'date','close','大盘价值')
juchao=pd.merge(temp, temp1, left_index=True, right_index=True, how='inner')

# 中盘成长价值
df=get_index_baostock('sz','399374')
temp=format_df(df,'date','close','中盘成长')
df=get_index_baostock('sz','399375')
temp1=format_df(df,'date','close','中盘价值')
juchao=pd.merge(juchao, temp, left_index=True, right_index=True, how='inner')
juchao=pd.merge(juchao, temp1, left_index=True, right_index=True, how='inner')

# 小盘成长价值
df=get_index_baostock('sz','399376')
temp=format_df(df,'date','close','小盘成长')
df=get_index_baostock('sz','399377')
temp1=format_df(df,'date','close','小盘价值')
juchao=pd.merge(juchao, temp, left_index=True, right_index=True, how='inner')
juchao=pd.merge(juchao, temp1, left_index=True, right_index=True, how='inner')
juchao


login success!
logout success!
login success!
logout success!
login success!
logout success!
login success!
logout success!
login success!
logout success!
login success!
logout success!


,大盘成长,大盘价值,中盘成长,中盘价值,小盘成长,小盘价值
date,,,,,,
2003-01-03,-0.003504,0.000444,-0.002889,-0.001058,0.000807,0.000777
2003-01-06,0.008238,0.009455,0.011741,0.012600,0.013549,0.016441
2003-01-07,-0.002385,-0.003362,-0.001837,0.000340,-0.000151,-0.002642
2003-01-08,0.027819,0.033120,0.026263,0.027113,0.026249,0.028295
2003-01-09,0.016357,0.016965,0.019018,0.018258,0.018617,0.019647
...,...,...,...,...,...,...
2026-01-30,-0.008011,-0.008618,-0.025617,-0.027294,-0.012213,-0.016467
2026-02-02,-0.021875,-0.013232,-0.039446,-0.045293,-0.035924,-0.041743
2026-02-03,0.012986,-0.000677,0.031577,0.024537,0.035780,0.021769


### 行业数据

#### 获取行业代码，批量获取行业数据

In [ ]:
industry_data_dict = {}

sw=get_sw_first_info()
print("遍历行业代码和行业名称:")
for index, row in sw.iterrows():
    industry_code = row['行业代码']
    industry_name = row['行业名称']
    industry_code = industry_code.replace('.SI', '') # Clean the code by removing '.SI'
    print(f"行业代码: {industry_code}, 行业名称: {industry_name}")
    df=get_sw_first_induesty_data(industry_code)
    industry_data_dict[industry_name]=df
industry_data_dict

遍历行业代码和行业名称:
行业代码: 801010, 行业名称: 农林牧渔
行业代码: 801030, 行业名称: 基础化工
行业代码: 801040, 行业名称: 钢铁
行业代码: 801050, 行业名称: 有色金属
行业代码: 801080, 行业名称: 电子
行业代码: 801880, 行业名称: 汽车
行业代码: 801110, 行业名称: 家用电器
行业代码: 801120, 行业名称: 食品饮料
行业代码: 801130, 行业名称: 纺织服饰
行业代码: 801140, 行业名称: 轻工制造
行业代码: 801150, 行业名称: 医药生物
行业代码: 801160, 行业名称: 公用事业
行业代码: 801170, 行业名称: 交通运输
行业代码: 801180, 行业名称: 房地产
行业代码: 801200, 行业名称: 商贸零售
行业代码: 801210, 行业名称: 社会服务
行业代码: 801780, 行业名称: 银行
行业代码: 801790, 行业名称: 非银金融
行业代码: 801230, 行业名称: 综合
行业代码: 801710, 行业名称: 建筑材料
行业代码: 801720, 行业名称: 建筑装饰
行业代码: 801730, 行业名称: 电力设备
行业代码: 801890, 行业名称: 机械设备
行业代码: 801740, 行业名称: 国防军工
行业代码: 801750, 行业名称: 计算机
行业代码: 801760, 行业名称: 传媒
行业代码: 801770, 行业名称: 通信
行业代码: 801950, 行业名称: 煤炭
行业代码: 801960, 行业名称: 石油石化
行业代码: 801970, 行业名称: 环保
行业代码: 801980, 行业名称: 美容护理


{'农林牧渔':           代码          日期       收盘       开盘       最高       最低        成交量  \
 0     801010  1999-12-30  1000.00  1000.00  1000.00  1000.00   0.098647   
 1     801010  2000-01-04  1027.66  1001.98  1035.36   985.47   0.140116   
 2     801010  2000-01-05  1028.87  1035.47  1057.85  1013.07   0.253309   
 3     801010  2000-01-06  1065.67  1025.38  1070.79  1011.00   0.302028   
 4     801010  2000-01-07  1106.19  1076.06  1117.71  1058.83   0.648305   
 ...      ...         ...      ...      ...      ...      ...        ...   
 6303  801010  2026-01-29  2957.64  2910.82  2957.64  2910.82  43.526770   
 6304  801010  2026-01-30  2976.84  2947.36  3034.01  2947.36  59.956488   
 6305  801010  2026-02-02  2872.03  2933.95  2933.95  2870.77  47.193033   
 6306  801010  2026-02-03  2898.79  2879.92  2899.03  2853.41  30.911429   
 6307  801010  2026-02-04  2936.45  2912.38  2938.00  2885.40  35.020210   
 
              成交额  
 0       1.239666  
 1       1.831998  
 2       2.904446 

#### 合并行业数据

In [ ]:
industry_returns_list = []
for industry_name, df  in industry_data_dict.items():
    if not df.empty: # Changed from df_1000.empty to df.empty, as df_1000 is not relevant here
        # Rename '日期' column to 'date'
        df.rename(columns={'日期': 'date'}, inplace=True)

        # Convert 'date' to datetime and set as index
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date').sort_index()

        # Calculate daily percentage change of '收盘' price
        returns_series = df['收盘'].pct_change().dropna()

        # Rename the series with the industry name
        returns_series = returns_series.rename(industry_name)

        # Append to the list
        industry_returns_list.append(returns_series)
    else:
        print(f"Warning: DataFrame for industry '{industry_name}' is empty. Skipping return calculation.")


merged_industry = pd.concat(industry_returns_list, axis=1, join='inner')

merged_industry

,农林牧渔,基础化工,钢铁,有色金属,电子,汽车,家用电器,食品饮料,纺织服饰,轻工制造,...,电力设备,机械设备,国防军工,计算机,传媒,通信,煤炭,石油石化,环保,美容护理
date,,,,,,,,,,,,,,,,,,,,,
2021-12-14,-0.000124,-0.006824,-0.018326,-0.023189,0.000020,-0.013940,-0.019551,0.005925,-0.004129,0.001312,...,-0.006945,-0.000011,0.012858,0.013403,0.010491,0.010581,-0.022833,-0.016406,0.007203,-0.008181
2021-12-15,-0.009176,-0.008445,0.007726,-0.015046,-0.009533,-0.005412,-0.008937,-0.014082,-0.000857,0.000724,...,0.002118,-0.003392,-0.005752,-0.004900,0.000774,0.003292,0.013745,-0.001826,0.009542,-0.014685
2021-12-16,-0.003147,0.009226,0.029313,0.007128,0.002004,0.004195,0.001637,-0.007722,0.002506,0.004548,...,0.005801,0.014111,0.010232,0.004757,0.016950,0.001487,0.063007,0.017339,0.023421,0.008354
2021-12-17,-0.001294,-0.013227,-0.004426,-0.010923,-0.022596,-0.019006,-0.017294,-0.025145,-0.002820,-0.005745,...,-0.022660,-0.012876,-0.017343,-0.012333,-0.003198,-0.007817,-0.020307,-0.001127,-0.001800,-0.027292
2021-12-20,-0.002049,-0.020304,-0.013609,-0.024530,-0.022938,-0.014438,-0.000516,0.001657,-0.003714,-0.009619,...,-0.042707,-0.013695,-0.023939,-0.015622,-0.026969,-0.022607,-0.022936,-0.020386,-0.012512,-0.018090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-29,0.014569,-0.009004,0.002195,0.016596,-0.035629,-0.014077,0.009195,0.065742,-0.003960,0.001969,...,-0.017834,-0.017643,-0.017929,0.000051,0.035278,-0.009815,0.003992,0.022264,-0.009302,0.015265
2026-01-30,0.006492,-0.003107,-0.028254,-0.077986,0.006845,-0.003965,-0.004702,-0.027817,-0.003281,0.005360,...,-0.004914,0.002112,-0.015515,-0.019596,-0.001724,0.034403,0.000954,-0.005441,-0.005593,-0.015665
2026-02-02,-0.035208,-0.056854,-0.059267,-0.076195,-0.040430,-0.024716,-0.004872,0.011083,-0.024095,-0.025164,...,-0.008025,-0.019195,-0.017375,-0.026711,-0.021888,-0.028894,-0.056432,-0.055097,-0.019458,-0.007042


### 黄金数据获取


In [ ]:
df=get_gold('Au99.99')
gold_df=format_df(df,'date','close','gold')
# gold_df['date'] = pd.to_datetime(gold_df['date']) # Ensure it's datetime
# gold_df = gold_df.set_index('date').sort_index() # Set date as index
# # Assuming return column is 'close' and date is already handled as index
# gold_df['gold'] = gold_df['close'].pct_change()
# gold_df = gold_df['gold'].dropna()
gold_df


Attempt 1 failed for get_gold with args ('Au99.99',), kwargs {}: HTTPSConnectionPool(host='www.sge.com.cn', port=443): Max retries exceeded with url: /graph/Dailyhq (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7de19a4f98b0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))


,gold
date,
2016-12-20,-0.002664
2016-12-21,-0.004159
2016-12-22,-0.003717
2016-12-23,0.000269
2016-12-26,0.005114
...,...
2026-01-29,0.048997
2026-01-30,-0.063611
2026-02-02,-0.115082


### 铜的数据获取

In [ ]:
copper_data = yf.download("HG=F", start='2000-01-01')
copper_data.index = pd.to_datetime(copper_data.index)
copper_data.index.name = 'date'

# Calculate daily returns for copper and create a new DataFrame
copper_df = pd.DataFrame(index=copper_data.index)
copper_df['copper'] = copper_data['Close'].pct_change().dropna()
copper_df

/tmp/ipython-input-2706444824.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  copper_data = yf.download("HG=F", start='2000-01-01')
[*********************100%***********************]  1 of 1 completed


,copper
date,
2000-08-30,NaN
2000-08-31,0.000000
2000-09-01,0.004520
2000-09-05,0.019123
2000-09-06,-0.004967
...,...
2026-01-29,0.048027
2026-01-30,-0.045098
2026-02-02,-0.016195


### 合并行业收益率和因子数据

In [ ]:
# merge 沪深300 ，smb ，hml
merge=pd.merge(df_300, smb, left_index=True, right_index=True, how='inner')
merge=pd.merge(merge, hml, left_index=True, right_index=True, how='inner')
merge = pd.merge(merge,juchao , left_index=True, right_index=True, how='inner')

# merge 行业数据
merge = pd.merge(merged_industry, merge, left_index=True, right_index=True, how='inner')
# merge 黄金 铜
merge = pd.merge(merge,gold_df , left_index=True, right_index=True, how='inner')
merge = pd.merge(merge,copper_df , left_index=True, right_index=True, how='inner')
merge

,农林牧渔,基础化工,钢铁,有色金属,电子,汽车,家用电器,食品饮料,纺织服饰,轻工制造,...,通信,煤炭,石油石化,环保,美容护理,沪深300,SMB,HML,gold,copper
date,,,,,,,,,,,,,,,,,,,,,
2021-12-14,-0.000124,-0.006824,-0.018326,-0.023189,0.000020,-0.013940,-0.019551,0.005925,-0.004129,0.001312,...,0.010581,-0.022833,-0.016406,0.007203,-0.008181,-0.006709,0.006324,-0.005910,0.000299,-0.006075
2021-12-15,-0.009176,-0.008445,0.007726,-0.015046,-0.009533,-0.005412,-0.008937,-0.014082,-0.000857,0.000724,...,0.003292,0.013745,-0.001826,0.009542,-0.014685,-0.008673,0.005385,0.011048,-0.010037,-0.017513
2021-12-16,-0.003147,0.009226,0.029313,0.007128,0.002004,0.004195,0.001637,-0.007722,0.002506,0.004548,...,0.001487,0.063007,0.017339,0.023421,0.008354,0.005760,0.000575,0.006410,0.009370,0.028951
2021-12-17,-0.001294,-0.013227,-0.004426,-0.010923,-0.022596,-0.019006,-0.017294,-0.025145,-0.002820,-0.005745,...,-0.007817,-0.020307,-0.001127,-0.001800,-0.027292,-0.015884,0.001911,0.014369,0.012631,-0.002209
2021-12-20,-0.002049,-0.020304,-0.013609,-0.024530,-0.022938,-0.014438,-0.000516,0.001657,-0.003714,-0.009619,...,-0.022607,-0.022936,-0.020386,-0.012512,-0.018090,-0.015004,-0.003003,0.010897,-0.002661,-0.000699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-29,0.014569,-0.009004,0.002195,0.016596,-0.035629,-0.014077,0.009195,0.065742,-0.003960,0.001969,...,-0.009815,0.003992,0.022264,-0.009302,0.015265,0.007605,-0.015651,0.010389,0.048997,0.048027
2026-01-30,0.006492,-0.003107,-0.028254,-0.077986,0.006845,-0.003965,-0.004702,-0.027817,-0.003281,0.005360,...,0.034403,0.000954,-0.005441,-0.005593,-0.015665,-0.009998,0.000715,-0.003859,-0.063611,-0.045098
2026-02-02,-0.035208,-0.056854,-0.059267,-0.076195,-0.040430,-0.024716,-0.004872,0.011083,-0.024095,-0.025164,...,-0.028894,-0.056432,-0.055097,-0.019458,-0.007042,-0.021324,-0.012526,0.007701,-0.115082,-0.016195


### 保存因子数据，保存到github

In [ ]:
# factors_file_path = os.path.join(path, 'factors.parquet')
# merge.to_parquet(factors_file_path, index=True)
# print(f"Saved factors data to {factors_file_path}")

# 获取当前 notebook 所在的绝对路径
current_path = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()
file_path = os.path.join(current_path, 'factors.parquet')
print(current_path)
# 保存文件
merge.to_parquet(file_path, engine='pyarrow', index=False)
# 需要先安装: pip install pyarrow

/content


NameError: name 'merge' is not defined

## 基金数据获取

In [ ]:
fund_returns_list = []

for target in fund_list:
  code = target['code']
  name = target['name']
  print(f"Processing fund: {name} ({code})")
  try:
    df = get_fund_data(code, "累计净值走势")
    if not df.empty:
      fund_series = format_df(df, '净值日期', '累计净值', name)
      fund_returns_list.append(fund_series)
    else:
      print(f"Warning: No data returned for fund {name} ({code}). Skipping.")
  except Exception as e:
    print(f"Error processing fund {name} ({code}): {e}. Skipping.")

# Concatenate all fund return Series into a single DataFrame
if fund_returns_list:
    fund_returns_df = pd.concat(fund_returns_list, axis=1, join='outer')
    current_path = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()
    file_path = os.path.join(current_path, 'fund.parquet')
    print(current_path)
    # 保存文件
    fund_returns_df.to_parquet(file_path, engine='pyarrow', index=False)





Processing fund: 易方达瑞享混合I (001437)
Processing fund: 财通资管医疗保健混合A (018484)


,易方达瑞享混合I,财通资管医疗保健混合A
date,,
2015-06-30,0.000000,NaN
2015-07-03,0.000000,NaN
2015-07-08,0.000000,NaN
2015-07-09,0.000000,NaN
2015-07-10,0.211000,NaN
...,...,...
2026-01-30,0.052333,0.000761
2026-02-02,-0.020346,-0.036234
2026-02-03,0.033769,0.024965


# 获取a股所有的上市公司

In [30]:
stock_df=get_all_stock()

current_path = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()
file_path = os.path.join(current_path, 'stock.parquet')

stock_df.to_csv(file_path, engine='pyarrow',index=False)